# Predicting Disruption Likelihood Score

This notebook builds and evaluates regression models to predict `disruption_likelihood_score` using operational and contextual features from the expanded dataset.

Objectives:
- Prepare features with robust preprocessing (imputation, scaling, encoding)
- Train/evaluate multiple models via cross-validation and a holdout test
- Interpret key drivers via feature importance



In [5]:
# Setup
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

# Display settings
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 200)

sns.set_theme(style='whitegrid', context='notebook')
plt.rcParams['figure.figsize'] = [10, 6]
plt.rcParams['axes.grid'] = True

DATA_PATH = os.path.join('dynamic_supply_chain_with_modes_expanded.csv')
na_tokens = ['', ' ', 'na', 'n/a', 'NA', 'N/A', 'null', 'NULL', 'none', 'None', 'nan', 'NaN', 'NAN', '-', '--']

df = pd.read_csv(DATA_PATH, na_values=na_tokens, keep_default_na=True, low_memory=False)
print(f"Loaded {len(df):,} rows and {len(df.columns)} columns")


Loaded 32,065 rows and 30 columns


In [6]:
# Define target and features
TARGET = 'disruption_likelihood_score'

# Candidate features (exclude obvious leakage and identifiers)
candidate_features = [
    # operational
    'fuel_consumption_rate', 'eta_variation_hours', 'traffic_congestion_level',
    'warehouse_inventory_level', 'loading_unloading_time', 'handling_equipment_availability',
    'shipping_costs', 'lead_time_days', 'customs_clearance_time',
    'port_congestion_level', 'route_risk_level', 'speed_kmh',
    # geo-temporal context (if present)
    'vehicle_gps_latitude', 'vehicle_gps_longitude',
    # engineered/inferred
    'inferred_mode'
]

available_features = [c for c in candidate_features if c in df.columns]
missing_features = sorted(set(candidate_features) - set(available_features))
print('Using features:', available_features)
if missing_features:
    print('Missing (skipped):', missing_features)

# Filter rows with target available
mod_df = df.dropna(subset=[TARGET]).copy()

X = mod_df[available_features].copy()
y = mod_df[TARGET].astype(float).copy()

# Identify numeric and categorical columns
numeric_cols = [c for c in X.columns if pd.api.types.is_numeric_dtype(X[c])]
categorical_cols = [c for c in X.columns if c not in numeric_cols]
print('Numeric:', numeric_cols)
print('Categorical:', categorical_cols)



Using features: ['fuel_consumption_rate', 'eta_variation_hours', 'traffic_congestion_level', 'warehouse_inventory_level', 'loading_unloading_time', 'handling_equipment_availability', 'shipping_costs', 'lead_time_days', 'customs_clearance_time', 'port_congestion_level', 'route_risk_level', 'speed_kmh', 'vehicle_gps_latitude', 'vehicle_gps_longitude', 'inferred_mode']
Numeric: ['fuel_consumption_rate', 'eta_variation_hours', 'traffic_congestion_level', 'warehouse_inventory_level', 'loading_unloading_time', 'handling_equipment_availability', 'shipping_costs', 'lead_time_days', 'customs_clearance_time', 'port_congestion_level', 'route_risk_level', 'speed_kmh', 'vehicle_gps_latitude', 'vehicle_gps_longitude']
Categorical: ['inferred_mode']


In [7]:
# Preprocessing pipeline
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler(with_mean=True, with_std=True))
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('cat', categorical_transformer, categorical_cols)
    ]
)

# Helper to evaluate a model with CV and holdout
def evaluate_model(model, X, y, model_name, n_splits=5, random_state=42):
    pipe = Pipeline(steps=[('preprocess', preprocessor), ('model', model)])
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    
    # Cross-validated metrics (sklearn-compatible across versions)
    cv_mse_scores = -cross_val_score(pipe, X, y, scoring='neg_mean_squared_error', cv=kf)
    cv_rmse = np.sqrt(cv_mse_scores).mean()
    cv_mae = (-cross_val_score(pipe, X, y, scoring='neg_mean_absolute_error', cv=kf)).mean()
    cv_r2 = cross_val_score(pipe, X, y, scoring='r2', cv=kf).mean()
    
    # Holdout split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=random_state
    )
    pipe.fit(X_train, y_train)
    preds = pipe.predict(X_test)
    holdout_rmse = np.sqrt(mean_squared_error(y_test, preds))
    holdout_mae = mean_absolute_error(y_test, preds)
    holdout_r2 = r2_score(y_test, preds)
    
    results = {
        'model': model_name,
        'cv_rmse': cv_rmse,
        'cv_mae': cv_mae,
        'cv_r2': cv_r2,
        'holdout_rmse': holdout_rmse,
        'holdout_mae': holdout_mae,
        'holdout_r2': holdout_r2,
        'fitted_pipeline': pipe
    }
    return results



In [8]:
# Train and evaluate candidate models (favor faster boosting over RandomForest)
from sklearn.ensemble import HistGradientBoostingRegressor

models = [
    ('Ridge', Ridge(alpha=1.0, random_state=42)),
    ('GradientBoosting', GradientBoostingRegressor(
        n_estimators=400, learning_rate=0.05, max_depth=3, random_state=42
    )),
    ('HistGradientBoosting', HistGradientBoostingRegressor(
        learning_rate=0.06, max_leaf_nodes=31, min_samples_leaf=50, random_state=42
    ))
]

results = []
for name, model in models:
    res = evaluate_model(model, X, y, name)
    results.append(res)
    print(f"{name}: CV RMSE={res['cv_rmse']:.3f}, CV MAE={res['cv_mae']:.3f}, CV R2={res['cv_r2']:.3f} | "
          f"Holdout RMSE={res['holdout_rmse']:.3f}, MAE={res['holdout_mae']:.3f}, R2={res['holdout_r2']:.3f}")

# Results dataframe
eval_df = pd.DataFrame([{k: v for k, v in r.items() if k != 'fitted_pipeline'} for r in results])

display(eval_df.sort_values('holdout_rmse'))


Ridge: CV RMSE=0.267, CV MAE=0.213, CV R2=0.084 | Holdout RMSE=0.270, MAE=0.214, R2=0.087
GradientBoosting: CV RMSE=0.255, CV MAE=0.200, CV R2=0.163 | Holdout RMSE=0.259, MAE=0.200, R2=0.163
HistGradientBoosting: CV RMSE=0.253, CV MAE=0.197, CV R2=0.177 | Holdout RMSE=0.257, MAE=0.199, R2=0.174


,model,cv_rmse,cv_mae,cv_r2,holdout_rmse,holdout_mae,holdout_r2
2,HistGradientBoosting,0.253171,0.197471,0.177341,0.256870,0.199078,0.173627
1,GradientBoosting,0.255342,0.199678,0.163158,0.258500,0.200476,0.163102
0,Ridge,0.267089,0.212849,0.084379,0.269944,0.214209,0.087363


In [9]:
# Feature importance for tree models and coefficients for Ridge
best_result = min(results, key=lambda r: r['holdout_rmse'])
best_name = best_result['model']
best_pipe = best_result['fitted_pipeline']
print(f"Best model by holdout RMSE: {best_name}")

# Fit on full data for interpretation
best_pipe.fit(X, y)

# Get transformed feature names
num_features = numeric_cols
cat_features = list(best_pipe.named_steps['preprocess']
                    .named_transformers_['cat']
                    .named_steps['encoder']
                    .get_feature_names_out(categorical_cols)) if categorical_cols else []
feature_names = num_features + cat_features

if best_name in ['RandomForest', 'GradientBoosting']:
    model = best_pipe.named_steps['model']
    importances = getattr(model, 'feature_importances_', None)
    if importances is not None:
        imp_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
        imp_df = imp_df.sort_values('Importance', ascending=False).head(20)
        plt.figure(figsize=(10, 6))
        sns.barplot(data=imp_df, x='Importance', y='Feature')
        plt.title(f'Top 20 Feature Importances — {best_name}')
        plt.tight_layout()
        plt.show()
else:
    # Ridge coefficients
    coefs = best_pipe.named_steps['model'].coef_
    coef_df = pd.DataFrame({'Feature': feature_names, 'Coefficient': coefs})
    coef_df['abs_coef'] = coef_df['Coefficient'].abs()
    coef_df = coef_df.sort_values('abs_coef', ascending=False).head(20)
    plt.figure(figsize=(10, 6))
    sns.barplot(data=coef_df, x='abs_coef', y='Feature')
    plt.title(f'Top 20 Absolute Coefficients — {best_name}')
    plt.xlabel('Absolute Coefficient')
    plt.tight_layout()
    plt.show()

# Calibration/fit plot for best model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
y_pred = best_pipe.fit(X_train, y_train).predict(X_test)
plt.figure(figsize=(6, 6))
sns.scatterplot(x=y_test, y=y_pred, alpha=0.4)
lims = [0, 1]
plt.plot(lims, lims, 'r--', linewidth=1)
plt.xlim(lims); plt.ylim(lims)
plt.xlabel('Actual Disruption Likelihood')
plt.ylabel('Predicted Disruption Likelihood')
plt.title(f'Predicted vs Actual — {best_name}')
plt.tight_layout()
plt.show()


Best model by holdout RMSE: HistGradientBoosting


AttributeError: 'HistGradientBoostingRegressor' object has no attribute 'coef_'

## Notes
- Target range is assumed to be [0, 1]. RMSE/MAE should be interpreted in that scale.
- If you suspect leakage or want stricter evaluation, use a time-based split on `timestamp`.
- You can add other models (e.g., XGBoost, LightGBM) if available in your environment.



In [ ]:
# Advanced feature selection: mutual information to focus on signal
from sklearn.feature_selection import mutual_info_regression

# Work only on numeric + one-hot encoded categorical approximation for MI
mi_input = mod_df[numeric_cols].copy()
if categorical_cols:
    mi_input = pd.concat([mi_input, pd.get_dummies(mod_df[categorical_cols], drop_first=False)], axis=1)

mi_scores = mutual_info_regression(mi_input, y, random_state=42)
mi_df = pd.DataFrame({'Feature': mi_input.columns, 'MI': mi_scores}).sort_values('MI', ascending=False)

top_k = 20
top_features = mi_df.head(top_k)['Feature'].tolist()
print('Top features by mutual information:')
print(mi_df.head(top_k))

# Build a reduced X for models that benefit from focusing on stronger signal
X_mi = mi_input[top_features]



In [ ]:
# Nonlinear GBM with native handling of missing values and monotonicity-friendly splits
from sklearn.ensemble import HistGradientBoostingRegressor

hgb = HistGradientBoostingRegressor(
    learning_rate=0.06,
    max_depth=None,
    max_leaf_nodes=31,
    min_samples_leaf=50,
    l2_regularization=0.0,
    random_state=42
)

# Use MI-selected features for HGB (already numeric)
X_train_mi, X_test_mi, y_train, y_test = train_test_split(X_mi, y, test_size=0.2, random_state=42)
hgb.fit(X_train_mi, y_train)
y_pred_hgb = hgb.predict(X_test_mi)

hgb_rmse = np.sqrt(mean_squared_error(y_test, y_pred_hgb))
hgb_mae = mean_absolute_error(y_test, y_pred_hgb)
hgb_r2 = r2_score(y_test, y_pred_hgb)

print(f"HistGradientBoosting — Holdout RMSE={hgb_rmse:.3f}, MAE={hgb_mae:.3f}, R2={hgb_r2:.3f}")

# Plot permutation importances for interpretability
from sklearn.inspection import permutation_importance
perm = permutation_importance(hgb, X_test_mi, y_test, n_repeats=5, random_state=42, scoring='neg_mean_squared_error')
imp_df = pd.DataFrame({'Feature': X_mi.columns, 'Importance': perm.importances_mean}).sort_values('Importance', ascending=False).head(20)
plt.figure(figsize=(10, 6))
sns.barplot(data=imp_df, x='Importance', y='Feature')
plt.title('Permutation Importances — HistGradientBoosting (Top 20)')
plt.tight_layout()
plt.show()


In [ ]:
# Stacking ensemble to combine linear and boosting models
from sklearn.ensemble import StackingRegressor

base_estimators = [
    ('ridge', Ridge(alpha=1.0, random_state=42)),
    ('gbr', GradientBoostingRegressor(n_estimators=250, learning_rate=0.06, max_depth=3, random_state=42))
]

stack = StackingRegressor(
    estimators=base_estimators,
    final_estimator=HistGradientBoostingRegressor(learning_rate=0.06, max_leaf_nodes=31, min_samples_leaf=50, random_state=42),
    passthrough=False,
    n_jobs=-1
)

stack_pipe = Pipeline(steps=[('preprocess', preprocessor), ('model', stack)])
X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(X, y, test_size=0.2, random_state=42)
stack_pipe.fit(X_train_s, y_train_s)
yp_s = stack_pipe.predict(X_test_s)
print(f"Stacking — Holdout RMSE={np.sqrt(mean_squared_error(y_test_s, yp_s)):.3f}, MAE={mean_absolute_error(y_test_s, yp_s):.3f}, R2={r2_score(y_test_s, yp_s):.3f}")


In [ ]:
# Per-mode specialized models (to capture heterogeneity)
mode_models = {}
metrics_rows = []

for mode in sorted(mod_df['inferred_mode'].dropna().unique()):
    subset = mod_df[mod_df['inferred_mode'] == mode]
    if len(subset) < 100:  # skip tiny groups
        continue
    X_subset = subset[available_features].copy()
    y_subset = subset[TARGET].astype(float).copy()

    # Build a pipeline per mode
    pipe_mode = Pipeline(steps=[('preprocess', preprocessor), ('model', GradientBoostingRegressor(
        n_estimators=400, learning_rate=0.05, max_depth=3, random_state=42
    ))])
    X_tr, X_te, y_tr, y_te = train_test_split(X_subset, y_subset, test_size=0.2, random_state=42)
    pipe_mode.fit(X_tr, y_tr)
    yhat = pipe_mode.predict(X_te)

    rmse = np.sqrt(mean_squared_error(y_te, yhat))
    mae = mean_absolute_error(y_te, yhat)
    r2 = r2_score(y_te, yhat)
    metrics_rows.append({'mode': mode, 'rmse': rmse, 'mae': mae, 'r2': r2})
    mode_models[mode] = pipe_mode

pm_df = pd.DataFrame(metrics_rows).sort_values('rmse')
print('Per-mode model metrics:')
display(pm_df)



In [ ]:
# Optional: XGBoost and LightGBM (run if installed)
try:
    from xgboost import XGBRegressor
    xgb = XGBRegressor(
        n_estimators=500, learning_rate=0.05, max_depth=6, subsample=0.8, colsample_bytree=0.8,
        reg_lambda=1.0, objective='reg:squarederror', random_state=42, n_jobs=-1
    )
    # Use MI-selected features for speed
    X_tr, X_te, y_tr, y_te = train_test_split(X_mi, y, test_size=0.2, random_state=42)
    xgb.fit(X_tr, y_tr)
    yph = xgb.predict(X_te)
    print(f"XGBoost — Holdout RMSE={np.sqrt(mean_squared_error(y_te, yph)):.3f}, MAE={mean_absolute_error(y_te, yph):.3f}, R2={r2_score(y_te, yph):.3f}")
except Exception as e:
    print('XGBoost not run:', str(e))

try:
    from lightgbm import LGBMRegressor
    lgbm = LGBMRegressor(
        n_estimators=600, learning_rate=0.05, max_depth=-1, num_leaves=63,
        subsample=0.8, colsample_bytree=0.8, reg_lambda=0.0, random_state=42, n_jobs=-1
    )
    X_tr, X_te, y_tr, y_te = train_test_split(X_mi, y, test_size=0.2, random_state=42)
    lgbm.fit(X_tr, y_tr)
    yph = lgbm.predict(X_te)
    print(f"LightGBM — Holdout RMSE={np.sqrt(mean_squared_error(y_te, yph)):.3f}, MAE={mean_absolute_error(y_te, yph):.3f}, R2={r2_score(y_te, yph):.3f}")
except Exception as e:
    print('LightGBM not run:', str(e))
